# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
from sklearn.svm import SVC
from keras.utils import to_categorical
import warnings
import sklearn.exceptions

In [ ]:
warnings.filterwarnings('ignore', category=sklearn.exceptions.UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Load all Data Sets

In [ ]:
# Main X and Y data
x_train = pd.read_csv('Data/x_train_gr_smpl.csv')
x_test = pd.read_csv('Data/x_test_gr_smpl.csv')

y_train = pd.read_csv('Data/y_train_smpl.csv')
y_test = pd.read_csv('Data/y_test_smpl.csv')

# 4000 Training samples
x_4000 = x_train[:4000]
y_4000 = y_train[:4000]

# 9000 Training samples
x_9000 = x_train[:9000]
y_9000 = y_train[:9000]

x_train_4000 = x_train.drop(index=x_4000.index)
y_train_4000 = y_train.drop(index=y_4000.index)

x_test_4000 = x_test.append(x_4000, ignore_index=True)
y_test_4000 = y_test.append(y_4000, ignore_index=True)

x_train_9000 = x_train.drop(index=x_9000.index)
y_train_9000 = y_train.drop(index=y_9000.index)

x_test_9000 = x_test.append(x_9000, ignore_index=True)
y_test_9000 = y_test.append(y_9000, ignore_index=True)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train_4000.shape, x_test_4000.shape, y_train_4000.shape, y_test_4000.shape)
print(x_train_9000.shape, x_test_9000.shape, y_train_9000.shape, y_test_9000.shape)

# Create Linear Classifier Function

In [ ]:
def linear_classifier(x_train, y_train, x_test, y_test):
    
    y_train = np.array(y_train).flatten()
    y_test = y_test.values.flatten()
    
    model = SVC()
    model.fit(x_train, y_train)
    
    score = model.score(x_test, y_test)
    
    return print('Score of Linear Classifier:', score)

# Create Neural Network Function, Analyze and Visualize Results

In [ ]:
def neural_network(x_train, y_train, x_test, y_test, hidden_layers, neurons, l_rate, epochs):
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 48, 48)
    x_train = x_train/255

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 48, 48)
    x_test = x_test/255

    # Neural Network Model
    model = Sequential()
    model.add(Flatten(input_shape = [48,48]))
    
    for i in range(hidden_layers):
        model.add(Dense(units=neurons, activation='relu'))
    
    model.add(Dense(units=10, activation='softmax'))
    
    
    opt = SGD(learning_rate=l_rate) 
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=epochs)
    y_pred = np.argmax(model.predict(x_test, verbose=0), axis=-1)

    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('Results')
    # Analyzing the results
    # Model Accuracy
    print('\n')
    scores = model.evaluate(x_test, y_test)
    print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print('\n')
    
    
    
    # Confusion Matrix
    cm = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(cm)
    print('\n')
    
    # True Positive
    TP = np.diag(cm)
    # False Positive
    FP = cm.sum(axis=0) - TP
    # False Negative
    FN = cm.sum(axis=1) - TP
    # True Negative
    TN = cm.sum() - (FP+FN+TP)


    #True Positive Rate
    TPR = TP/(TP+FN)
    TPR_mean = np.round(TPR.mean(), 2)
    #False Positive Rate
    FPR = FP/(FP+TN)
    FPR_mean = np.round(FPR.mean(), 2)
    #Precision
    Precision = TP/(TP+FP)
    Precision_mean = np.round(Precision.mean(), 2)
    #Recall
    Recall = TP/(TP+FN)
    Recall_mean = np.round(Recall.mean(), 2)
    #F1 Measure
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    F1_mean = np.round(F1.mean(), 2)
    
    np.set_printoptions(precision=2)
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('TP rate: ', TPR)
    print('Mean TP rate: ', TPR_mean)
    print('FP rate: ', FPR)
    print('Mean FP rate: ', FPR_mean)
    print('Precision: ', Precision)
    print('Mean Precision: ', Precision_mean)
    print('Recall: ', Recall)
    print('Mean Recall: ', Recall_mean)
    print('F Measure: ', F1)
    print('Mean F Measure: ', F1_mean)
    print('\n')
    
    print('Classification Report:')
    print(metrics.classification_report(y_test, y_pred))
    
    # Visualizing the results
    # Heatmap
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('\n')
    print('Visualization of Results')
    plt.figure(figsize=(11,7))
    sns.heatmap(cm, annot=True, fmt=' .0f', square=True, cmap='Blues_r')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Heatmap of Confusion Matrix')
    
    plt.figure(figsize=(8,4))
    plt.scatter(Precision, Recall, c='b')
    plt.xlabel('Precsion')
    plt.ylabel('Recall')
    plt.title('Precision vs Recall Scatter Chart')
    plt.show()
    print('---------------------------------------------------------------------------------------------------------------')

# Create Convolutional Neural Network, Analyze and Visualize Results

In [ ]:
def conv_neural_network(x_train, y_train, x_test, y_test, l_rate, epochs, momentum):    
    
    x_train = np.array(x_train)
    x_train = x_train.reshape((len(x_train), 48, 48, 1))
    x_train = x_train/255

    x_test = np.array(x_test)
    x_test = x_test.reshape((len(x_test), 48, 48, 1))
    x_test = x_test/255

    
    y_train = to_categorical(y_train)
    y_test_ = y_test.copy()
    y_test = to_categorical(y_test)
    
    
    # Neural Network Model
    model = Sequential()

    model.add(Conv2D(32, (3,3), input_shape=(48,48,1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())
    model.add(Dense(100))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    opt = SGD(learning_rate=l_rate, momentum=momentum)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=epochs, batch_size=48)
    
    y_pred = np.argmax(model.predict(x_test, verbose=0), axis=-1)
    
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('Results')
    # Analyzing the results
    # Model Accuracy
    print('\n')
    scores = model.evaluate(x_test, y_test)
    print(f'{model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print('\n')
    
    # Confusion Matrix
    y_test = y_test_.copy()
    cm = metrics.confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(cm)
    print('\n')
    
    # True Positive
    TP = np.diag(cm)
    # False Positive
    FP = cm.sum(axis=0) - TP
    # False Negative
    FN = cm.sum(axis=1) - TP
    # True Negative
    TN = cm.sum() - (FP+FN+TP)


    #True Positive Rate
    TPR = TP/(TP+FN)
    TPR_mean = np.round(TPR.mean(), 2)
    #False Positive Rate
    FPR = FP/(FP+TN)
    FPR_mean = np.round(FPR.mean(), 2)
    #Precision
    Precision = TP/(TP+FP)
    Precision_mean = np.round(Precision.mean(), 2)
    #Recall
    Recall = TP/(TP+FN)
    Recall_mean = np.round(Recall.mean(), 2)
    #F1 Measure
    F1 = 2 * (Precision * Recall) / (Precision + Recall)
    F1_mean = np.round(F1.mean(), 2)
    
    np.set_printoptions(precision=2)
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('TP rate: ', TPR)
    print('Mean TP rate: ', TPR_mean)
    print('FP rate: ', FPR)
    print('Mean FP rate: ', FPR_mean)
    print('Precision: ', Precision)
    print('Mean Precision: ', Precision_mean)
    print('Recall: ', Recall)
    print('Mean Recall: ', Recall_mean)
    print('F Measure: ', F1)
    print('Mean F Measure: ', F1_mean)
    print('\n')
    
    print('Classification Report:')
    print(metrics.classification_report(y_test, y_pred))
    
    # Visualizing the results
    # Heatmap
    print('---------------------------------------------------------------------------------------------------------------')
    print('\n')
    print('\n')
    print('Visualization of Results')
    plt.figure(figsize=(11,7))
    sns.heatmap(cm, annot=True, fmt=' .0f', square=True, cmap='Greens')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Heatmap of Confusion Matrix')
    
    plt.figure(figsize=(8,4))
    plt.scatter(Precision, Recall, c='g')
    plt.xlabel('Precsion')
    plt.ylabel('Recall')
    plt.title('Precision vs Recall Scatter Chart')
    plt.show()
    print('---------------------------------------------------------------------------------------------------------------')

# Main Data Sets

# Linear Classifier

In [ ]:
linear_classifier(x_train, y_train, x_test, y_test)

# Multi-Layer Perceptron

# Varying the Neural Network Parameters

# Hidden Layers (Other Parameters remain Constant)

# 1 Hidden Layer

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=1, neurons=100, l_rate=0.01, epochs=5)

# 2 Hidden Layers

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# 4 Hidden Layers

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=4, neurons=100, l_rate=0.01, epochs=5)

# Neurons in Hidden Layers (Other Parameters remain Constant)

# 50 Neurons

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=50, l_rate=0.01, epochs=5)

# 100 Neurons

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# 150 Neurons

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=150, l_rate=0.01, epochs=5)       

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.005, epochs=5)

# Learning Rate = 0.01

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# Learning Rate = 0.02

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.02, epochs=5)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# Epochs = 10

In [ ]:
neural_network(x_train, y_train, x_test, y_test, hidden_layers=2, neurons=100, l_rate=0.01, epochs=10)

# Convolutional Neural Network

# Varying the Neural Network Parameters

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.005, epochs=5, momentum=0.9)

# Learning Rate = 0.01

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.01, epochs=5, momentum=0.9)

# Learning Rate = 0.02

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.02, epochs=5, momentum=0.9)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.01, epochs=5, momentum=0.9)

# Epochs = 10

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.01, epochs=10, momentum=0.9)

# Momentum (Other Parameters remain Constant)

# Momentum = 0.5

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.01, epochs=5, momentum=0.5)

# Momentum = 0.7

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.01, epochs=5, momentum=0.7)

# Momentum = 0.9

In [ ]:
conv_neural_network(x_train, y_train, x_test, y_test, l_rate=0.01, epochs=5, momentum=0.9)

# 4000 Data Sets

# Linear Classifier

In [ ]:
linear_classifier(x_train_4000, y_train_4000, x_test_4000, y_test_4000)

# Multi-Layer Perceptron

# Varying the Neural Network Parameters

# Hidden Layers (Other Parameters remain Constant)

# 1 Hidden Layer

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=1, neurons=100, l_rate=0.01, epochs=5)

# 2 Hidden Layers

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# 4 Hidden Layers

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=4, neurons=100, l_rate=0.01, epochs=5)

# Neurons in Hidden Layers (Other Parameters remain Constant)

# 50 Neurons

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=50, l_rate=0.01, epochs=5)

# 100 Neurons

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# 150 Neurons

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=150, l_rate=0.01, epochs=5)       

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.005, epochs=5)

# Learning Rate = 0.01

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# Learning Rate = 0.02

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.02, epochs=5)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# Epochs = 10

In [ ]:
neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=10)

# Convolutional Neural Network

# Varying the Neural Network Parameters

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.005, epochs=5, momentum=0.9)

# Learning Rate = 0.01

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.01, epochs=5, momentum=0.9)

# Learning Rate = 0.02

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.02, epochs=5, momentum=0.9)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.01, epochs=5, momentum=0.9)

# Epochs = 10

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.01, epochs=10, momentum=0.9)

# Momentum (Other Parameters remain Constant)

# Momentum = 0.5

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.01, epochs=5, momentum=0.5)

# Momentum = 0.7

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.01, epochs=5, momentum=0.7)

# Momentum = 0.9

In [ ]:
conv_neural_network(x_train_4000, y_train_4000, x_test_4000, y_test_4000, l_rate=0.01, epochs=5, momentum=0.9)

# 9000 Data Sets 

# Linear Classifier

In [ ]:
linear_classifier(x_train_9000, y_train_9000, x_test_9000, y_test_9000)

# Multi-Layer Perceptron

# Varying the Neural Network Parameters

# Hidden Layers (Other Parameters remain Constant)

# 1 Hidden Layer

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=1, neurons=100, l_rate=0.01, epochs=5)

# 2 Hidden Layers

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# 4 Hidden Layers

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=4, neurons=100, l_rate=0.01, epochs=5)

# Neurons in Hidden Layers (Other Parameters remain Constant)

# 50 Neurons

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=50, l_rate=0.01, epochs=5)

# 100 Neurons

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# 150 Neurons

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=150, l_rate=0.01, epochs=5)       

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.005, epochs=5)

# Learning Rate = 0.01

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# Learning Rate = 0.02

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.02, epochs=5)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=5)

# Epochs = 10

In [ ]:
neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, hidden_layers=2, neurons=100, l_rate=0.01, epochs=10)

# Convolutional Neural Network

# Varying the Neural Network Parameters

# Learning Rate (Other Parameters remian Constant)

# Learning Rate = 0.005

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.005, epochs=5, momentum=0.9)

# Learning Rate = 0.01

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.01, epochs=5, momentum=0.9)

# Learning Rate = 0.02

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.02, epochs=5, momentum=0.9)

# Epochs (Other Parameters remain Constant)

# Epochs = 5

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.01, epochs=5, momentum=0.9)

# Epochs = 10

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.01, epochs=10, momentum=0.9)

# Momentum (Other Parameters remain Constant)

# Momentum = 0.5

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.01, epochs=5, momentum=0.5)

# Momentum = 0.7

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.01, epochs=5, momentum=0.7)

# Momentum = 0.9

In [ ]:
conv_neural_network(x_train_9000, y_train_9000, x_test_9000, y_test_9000, l_rate=0.01, epochs=5, momentum=0.9)